
# DOWNLOAD THE TEXT CORPUS

In [1]:
!wget http://mattmahoney.net/dc/text8.zip -O text8.gz
!gzip -d text8.gz -f

--2021-08-19 04:47:48--  http://mattmahoney.net/dc/text8.zip
Resolving mattmahoney.net (mattmahoney.net)... 67.195.197.24
Connecting to mattmahoney.net (mattmahoney.net)|67.195.197.24|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31344016 (30M) [application/zip]
Saving to: ‘text8.gz’

text8.gz            100%[===================>]  29.89M   728KB/s    in 43s     

2021-08-19 04:48:32 (713 KB/s) - ‘text8.gz’ saved [31344016/31344016]



# Train word2vec

In [2]:
!pip install gensim

In [3]:
from gensim import utils

with open('/content/text8','r') as file:
  documents = file.readlines()

class MyCorpus(object):
    def __iter__(self): 
        for line in documents:
            yield utils.simple_preprocess(line)

In [4]:
import gensim.models
sentences = MyCorpus()
cbowModel = gensim.models.Word2Vec(sentences=sentences,
                                       sg = 0,
                                       window = 5,
                                       size=100,
                                       min_count = 5,
                                       workers=16,
                                       iter = 15)

In [5]:
len(cbowModel.wv.vocab.keys())

70965

In [6]:
# get vector for a word
cbowModel.wv.get_vector('animal')

array([-2.4298667e-03, -4.2657708e-03, -3.9093681e-03, -3.3085307e-03,
       -2.6158961e-03,  2.0800901e-03,  1.9386655e-03, -6.5961218e-04,
        8.2291418e-04,  1.7548635e-03, -4.9275747e-03,  2.0899093e-03,
        3.7480083e-03, -7.3822885e-04, -4.9511134e-03, -5.8009813e-04,
        2.9283524e-03, -4.1930345e-03,  4.9804808e-03, -4.9215951e-03,
        1.9582943e-03,  1.6565860e-03,  2.0411178e-03, -1.6194192e-03,
        4.9168644e-03,  1.3057461e-03, -7.2616632e-07,  3.1212068e-03,
        9.8353415e-04, -2.0762733e-03, -7.0091017e-04,  4.2846319e-03,
       -7.6495239e-04, -2.5356936e-03, -3.9251042e-03, -6.0309545e-04,
        3.8825937e-03, -1.1505418e-03, -3.7038028e-03, -1.5265662e-03,
        3.3959455e-03, -4.3527279e-03,  1.5491099e-03,  2.6703142e-03,
       -3.0685626e-03,  4.5518707e-03,  2.1588446e-03,  2.3295549e-03,
        3.6848411e-03, -7.8449445e-04,  2.7258436e-03, -4.7434820e-03,
       -3.8107622e-03,  3.7402397e-03, -4.5044720e-03, -1.1315736e-03,
      

In [7]:
cbowModel.wv.most_similar('animal',topn=4)

[('poached', 0.3875291645526886),
 ('anakkale', 0.3805159628391266),
 ('fe', 0.37596219778060913),
 ('factbook', 0.3714029788970947)]

# Pretrained word2vec model

In [8]:
import gensim.downloader as api

model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


# word similarity

In [9]:
!pip install sklearn

Importing Data

RSS = Reference similarity score

In [10]:
import pandas as pd
import csv
data=pd.read_csv('/content/wordsim_similarity_goldstandard.txt',sep='\t')
data=pd.DataFrame(data)
data.columns=['Word1','Word2','RSS']
data.head(20)

,Word1,Word2,RSS
0,tiger,tiger,10.00
1,plane,car,5.77
2,train,car,6.31
3,television,radio,6.77
4,media,radio,7.42
5,bread,butter,6.19
6,cucumber,potato,5.92
7,doctor,nurse,7.00
8,professor,doctor,6.62
9,student,professor,6.81


In [11]:
from sklearn.metrics.pairwise import cosine_similarity
#function to get similarity
def get_word_similarity(word_1,word_2,model):
  word_1_vector = model.get_vector(word_1)
  word_2_vector = model.get_vector(word_2)
  similarity = cosine_similarity(word_1_vector.reshape(1,-1),word_2_vector.reshape(1,-1))[0][0]
  return similarity


for index,row in data.iterrows(): 
  data['CSS']=(get_word_similarity(row['Word1'],row['Word2'],model)) #CSS= COSINE SIMILARITY SCORE
data.head()

,Word1,Word2,RSS,CSS
0,tiger,tiger,10.00,0.118132
1,plane,car,5.77,0.118132
2,train,car,6.31,0.118132
3,television,radio,6.77,0.118132
4,media,radio,7.42,0.118132



## Spearman correlation

In [14]:
from scipy import stats
from scipy.stats.mstats import spearmanr

CSS=data['CSS']
RSS=data['RSS']

for index,row in data.iterrows(): 
  p = stats.spearmanr(CSS,RSS)
  print(p)

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrR

# Train fast text

In [15]:
!pip install fasttext

     |████████████████████████████████| 68 kB 4.7 MB/s 
  Using cached pybind11-2.7.1-py2.py3-none-any.whl (200 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3095893 sha256=acbd0350d433f456e9c811ab95288e490632e9f0c3ef80fd2896ea296ea15b8b
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [16]:
import fasttext
fassttext_model = fasttext.train_unsupervised(input = '/content/text8',
                                              ws = 5,
                                              model = 'cbow',
                                              dim = 100,
                                              epoch = 4)


## word similarity

In [21]:
for index,row in data.iterrows(): 
  data['CSS2']=get_word_similarity(row['Word1'],row['Word2'],model)
data.head()

,Word1,Word2,RSS,CSS,CSS_2,CSS2
0,tiger,tiger,10.00,0.118132,0.118132,0.118132
1,plane,car,5.77,0.118132,0.118132,0.118132
2,train,car,6.31,0.118132,0.118132,0.118132
3,television,radio,6.77,0.118132,0.118132,0.118132
4,media,radio,7.42,0.118132,0.118132,0.118132



## Spearman correlation

In [22]:
CSS=data['CSS2']
RSS=data['RSS']

for index,row in data.iterrows(): 
  p = stats.spearmanr(CSS,RSS)
  print(p)

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrR